<a href="https://colab.research.google.com/github/vincetofy223/federated-privacy-ml-paper/blob/main/federated_learning_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>